# Talkset 비윤리 강도 Regression 모델 학습 (KoBERT 기반)

Talkset 데이터셋의 `intensity` (비윤리 강도) 값을 예측하는 KoBERT 기반 Regression 모델을 학습합니다.

## Talkset 데이터 구조

Talkset 데이터셋에는 다음 필드가 있습니다:
- `is_immoral`: boolean (윤리적 문제 여부)
- `intensity`: float (비윤리 강도 점수, 0.0 ~ 약 2.0)
- `intensity_sum`: int (강도 합계)
- `types`: list (비윤리 유형, 예: "HATE", "CENSURE" 등)
- `text`: str (발화 텍스트)

**학습 목표**: 발화 텍스트를 입력받아 비윤리 강도(intensity)를 예측하는 Regression 모델 학습


## 1. 환경 설정 및 라이브러리 설치

Colab에서 실행 시 첫 번째 셀에서 라이브러리를 설치합니다.


In [13]:
# 필요한 라이브러리 설치
!pip install transformers torch numpy pandas scikit-learn tqdm

# GPU 확인
import torch
print(f'CUDA 사용 가능: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')


CUDA 사용 가능: False


In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 2. 라이브러리 Import 및 설정

Python 라이브러리를 import하고 기본 설정을 합니다.


In [26]:
import json
import torch
import numpy as np
import pandas as pd
from pathlib import Path
from typing import List, Dict, Tuple
from tqdm import tqdm

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import warnings
warnings.filterwarnings('ignore')

# GPU 사용 가능 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'사용 장치: {device}')
print('라이브러리 import 완료')


사용 장치: cpu
라이브러리 import 완료


## 3. Talkset 데이터 구조 확인

Talkset 데이터셋에는 다음 필드가 있습니다:
- `is_immoral`: boolean (윤리적 문제 여부)
- `intensity`: float (비윤리 강도 점수, 0.0 ~ 약 2.0) - **이것이 예측 대상**
- `intensity_sum`: int (강도 합계)
- `types`: list (비윤리 유형, 예: "HATE", "CENSURE" 등)
- `text`: str (발화 텍스트) - **이것이 입력**

**비윤리 강도 학습 체계:**
- Talkset 데이터셋에 `intensity` 필드가 이미 구현되어 있음
- 다수의 평가자(voter)가 각 문장의 비윤리 강도를 평가
- `intensity`는 평균 강도 값 (0.0 ~ 약 2.0)
- `intensity_sum`은 모든 평가자의 강도 합계
- `is_immoral=True`인 문장은 `intensity > 0`을 가짐

**⚠️ Colab 12시간 제약 대응:**
- 전체 데이터를 여러 배치로 나누어 증분 학습
- 각 배치마다 이전 모델을 로드하여 계속 fine-tuning
- 체크포인트 저장 및 재개 기능으로 중단 후 이어서 학습 가능

In [28]:
# 데이터 파일 확인 (Colab에서는 파일 업로드 필요)
# !wget https://example.com/talksets-train-6.json  # 또는 파일 직접 업로드
# from google.colab import files
# uploaded = files.upload()

# 샘플 데이터 확인
data_file = '/content/drive/MyDrive/ethic_datasets/talksets-train-6.json'  # 실제 경로로 수정

try:
    with open(data_file, 'r', encoding='utf-8') as f:
        data = json.load(f)

    print(f'총 대화 수: {len(data)}')

    # 첫 번째 비윤리 문장 찾기
    for talk in data[:10]:
        for sentence in talk.get('sentences', []):
            if sentence.get('is_immoral', False):
                print(f'\n샘플 비윤리 문장:')
                print(f'  텍스트: {sentence.get("text", "")}')
                print(f'  is_immoral: {sentence.get("is_immoral")}')
                print(f'  intensity: {sentence.get("intensity")}')
                print(f'  intensity_sum: {sentence.get("intensity_sum")}')
                print(f'  types: {sentence.get("types", [])}')
                break
        else:
            continue
        break

    # 통계 계산
    all_sentences = [s for talk in data for s in talk.get('sentences', [])]
    immoral_sentences = [s for s in all_sentences if s.get('is_immoral', False)]

    print(f'\n전체 문장 수: {len(all_sentences):,}')
    print(f'비윤리 문장 수: {len(immoral_sentences):,}')

    if immoral_sentences:
        intensities = [s.get('intensity', 0.0) for s in immoral_sentences]
        print(f'비윤리 문장 Intensity 통계:')
        print(f'  평균: {np.mean(intensities):.3f}')
        print(f'  최소: {np.min(intensities):.3f}')
        print(f'  최대: {np.max(intensities):.3f}')
        print(f'  표준편차: {np.std(intensities):.3f}')

except FileNotFoundError:
    print(f'파일을 찾을 수 없습니다: {data_file}')
    print('Colab에서는 파일을 업로드하거나 경로를 수정하세요.')


총 대화 수: 13298

샘플 비윤리 문장:
  텍스트: 원래 틀딱들은 눈치가 없어서 ㅋㅋㅋ
  is_immoral: True
  intensity: 1.4
  intensity_sum: 7
  types: ['CENSURE', 'HATE']

전체 문장 수: 45,215
비윤리 문장 수: 24,871
비윤리 문장 Intensity 통계:
  평균: 1.450
  최소: 1.000
  최대: 3.000
  표준편차: 0.380


## 4. 데이터 로드 및 증분 학습 설정

**증분 학습 전략:**
1. 전체 데이터를 여러 배치(예: 4개)로 나누기
2. 각 배치마다 이전에 학습한 모델을 로드하여 계속 fine-tuning
3. 각 배치 학습 후 체크포인트 저장 (Google Drive 권장)
4. 다음 세션에서 마지막 체크포인트 로드하여 이어서 학습

**장점:**
- Colab 12시간 제약 내에서 학습 가능
- 각 배치마다 모델이 점진적으로 개선됨
- 중단 후 재개 가능


In [30]:
def load_talkset_data(file_path: str, filter_immoral_only: bool = False) -> Tuple[List[str], List[float]]:    """    Talkset 데이터 로드    Args:        file_path: talkset JSON 파일 경로        filter_immoral_only: True이면 is_immoral=True인 문장만 로드    Returns:        (texts, intensities) 튜플    """    print(f'데이터 로드 중: {file_path}')    with open(file_path, 'r', encoding='utf-8') as f:        data = json.load(f)    texts = []    intensities = []    for talk in tqdm(data, desc='대화 처리 중'):        for sentence in talk.get('sentences', []):            text = sentence.get('text', '').strip()            is_immoral = sentence.get('is_immoral', False)            intensity = sentence.get('intensity', 0.0)            # 빈 텍스트 제외            if not text:                continue            # 필터링 조건            if filter_immoral_only and not is_immoral:                continue            texts.append(text)            intensities.append(intensity)    return texts, intensitiesdef load_multiple_talkset_files(file_paths: List[str], filter_immoral_only: bool = False) -> Tuple[List[str], List[float]]:    """    여러 Talkset 파일을 병합하여 로드    Args:        file_paths: talkset JSON 파일 경로 리스트        filter_immoral_only: True이면 is_immoral=True인 문장만 로드    Returns:        (texts, intensities) 튜플    """    all_texts = []    all_intensities = []    for file_path in file_paths:        texts, intensities = load_talkset_data(file_path, filter_immoral_only)        all_texts.extend(texts)        all_intensities.extend(intensities)        print(f'  파일 {file_path}: {len(texts):,}개 문장 로드 완료')    return all_texts, all_intensitiesdef split_data_into_batches(    texts: List[str],    intensities: List[float],    num_batches: int = 4,    shuffle: bool = True,    random_state: int = 42) -> List[Tuple[List[str], List[float]]]:    """    데이터를 여러 배치로 나누기    Args:        texts: 텍스트 리스트        intensities: intensity 리스트        num_batches: 배치 개수        shuffle: 셔플 여부        random_state: 랜덤 시드    Returns:        [(texts_batch1, intensities_batch1), ...] 리스트    """    if shuffle:        np.random.seed(random_state)        indices = np.random.permutation(len(texts))        texts = [texts[i] for i in indices]        intensities = [intensities[i] for i in indices]    batch_size = len(texts) // num_batches    batches = []    for i in range(num_batches):        start_idx = i * batch_size        if i == num_batches - 1:  # 마지막 배치는 나머지 모두 포함            end_idx = len(texts)        else:            end_idx = (i + 1) * batch_size        batch_texts = texts[start_idx:end_idx]        batch_intensities = intensities[start_idx:end_idx]        batches.append((batch_texts, batch_intensities))        print(f'  배치 {i+1}: {len(batch_texts):,}개 문장')    return batches# ===== 증분 학습 설정 =====# ⚠️ Talkset 데이터 파일 경로를 설정해주세요. ⚠️# 파일이 Colab 환경에 존재하지 않아 FileNotFoundError가 발생했습니다.# 아래 가이드를 참고하여 파일 경로를 정확히 설정한 후 다시 실행해주세요.# --- 방법 1: Google Drive에 파일이 있는 경우 (권장) ---# 1. '2. 라이브러리 Import 및 설정' 섹션에서 drive.mount('/content/drive') 코드를 실행하여 Drive를 마운트합니다.# 2. 아래 'TALKSET_DATA_DIR' 변수를 Google Drive 내 Talkset 데이터 폴더 경로로 설정합니다.#    예시: TALKSET_DATA_DIR = '/content/drive/MyDrive/talkset_data'# --- 방법 2: Colab 환경에 직접 파일 업로드 ---# 1. 좌측 파일 탐색기 아이콘 (폴더 모양)을 클릭하여 '파일' 탭을 엽니다.# 2. '세션 저장용량에 업로드' 버튼 (위로 향하는 화살표 아이콘)을 클릭하여 talkset JSON 파일들을 업로드합니다.# 3. 이 경우 TALKSET_DATA_DIR를 현재 작업 디렉토리 ('.')로 설정하거나, 파일명만 직접 나열합니다.#    예시: TALKSET_DATA_DIR = '.' 또는 아래에서 파일명만 설정.# --- 파일 경로 설정 ---# TODO: 아래 TALKSET_DATA_DIR와 TRAIN_DATA_FILES, VAL_DATA_FILE을 실제 경로에 맞게 수정하세요.TALKSET_DATA_DIR = '/content/drive/MyDrive/ethic_datasets' # 예를 들어, '/content/drive/MyDrive/talkset_data' 로 변경 가능TRAIN_DATA_FILES = [    f'{TALKSET_DATA_DIR}/talksets-train-1.json',    f'{TALKSET_DATA_DIR}/talksets-train-2.json',    f'{TALKSET_DATA_DIR}/talksets-train-3.json',    f'{TALKSET_DATA_DIR}/talksets-train-4.json',    f'{TALKSET_DATA_DIR}/talksets-train-5.json']VAL_DATA_FILE = f'{TALKSET_DATA_DIR}/talksets-train-6.json'# 데이터 배치 개수 (Colab 시간 제약에 따라 조정: 3~5개 권장)NUM_BATCHES = 10# 현재 학습할 배치 인덱스 (0부터 시작, 처음에는 0)# 이전 세션에서 학습이 완료된 경우 다음 배치 인덱스로 설정CURRENT_BATCH_INDEX = 0  # 0, 1, 2, 3... 순서대로 학습# 이전 모델 경로 (None이면 base model부터 시작)# 이전 세션에서 저장한 체크포인트 경로로 설정RESUME_FROM_CHECKPOINT = None  # 예: './kobert_immorality_intensity_regression/checkpoint-XXXX'print('='*60)print('증분 학습 설정')print('='*60)print(f'Training 파일: {len(TRAIN_DATA_FILES)}개')print(f'Validation 파일: {VAL_DATA_FILE}')print(f'총 배치 개수: {NUM_BATCHES}')print(f'현재 배치 인덱스: {CURRENT_BATCH_INDEX}')print(f'체크포인트에서 재개: {RESUME_FROM_CHECKPOINT if RESUME_FROM_CHECKPOINT else "아니오 (base model부터)"}')print('='*60)# Training 데이터 로드print('\n[Training 데이터 로드]')train_texts, train_intensities = load_multiple_talkset_files(    TRAIN_DATA_FILES,    filter_immoral_only=False)print(f'\n총 Training 문장 수: {len(train_texts):,}')print(f'  - intensity > 0: {sum(1 for i in train_intensities if i > 0):,}개')print(f'  - intensity = 0: {sum(1 for i in train_intensities if i == 0):,}개')# Training 데이터를 배치로 나누기print(f'\n[Training 데이터를 {NUM_BATCHES}개 배치로 분할]')train_batches = split_data_into_batches(    train_texts,    train_intensities,    num_batches=NUM_BATCHES,    shuffle=True,    random_state=42)# Validation 데이터 로드 (전체 데이터)print('\n[Validation 데이터 로드]')val_texts_full, val_intensities_full = load_talkset_data(    VAL_DATA_FILE,    filter_immoral_only=False)print(f'총 Validation 문장 수 (전체): {len(val_texts_full):,}')# 학습 중 체크포인트용 Validation 데이터 샘플링 (4,000개)VAL_SAMPLE_SIZE = 4000  # 학습 중 체크포인트용 validation 크기if len(val_texts_full) > VAL_SAMPLE_SIZE:    # 랜덤 샘플링 (stratified by intensity > 0)    immoral_indices = [i for i, intensity in enumerate(val_intensities_full) if intensity > 0]    normal_indices = [i for i, intensity in enumerate(val_intensities_full) if intensity == 0]        # 비율 유지하며 샘플링    immoral_sample_size = min(int(VAL_SAMPLE_SIZE * len(immoral_indices) / len(val_intensities_full)), len(immoral_indices))    normal_sample_size = VAL_SAMPLE_SIZE - immoral_sample_size    normal_sample_size = min(normal_sample_size, len(normal_indices))        np.random.seed(42)    sampled_immoral = np.random.choice(immoral_indices, immoral_sample_size, replace=False)    sampled_normal = np.random.choice(normal_indices, normal_sample_size, replace=False)        sampled_indices = np.concatenate([sampled_immoral, sampled_normal])    np.random.shuffle(sampled_indices)        val_texts = [val_texts_full[i] for i in sampled_indices]    val_intensities = [val_intensities_full[i] for i in sampled_indices]        print(f'  → 학습용 Validation (샘플링): {len(val_texts):,}개')    print(f'    - intensity > 0: {sum(1 for i in val_intensities if i > 0):,}개')    print(f'    - intensity = 0: {sum(1 for i in val_intensities if i == 0):,}개')else:    val_texts = val_texts_full    val_intensities = val_intensities_full    print(f'  → 전체 데이터 사용 (샘플 수가 {VAL_SAMPLE_SIZE}개 이하)')print(f'\nIntensity 통계:')print(f'  평균: {np.mean(train_intensities + val_intensities_full):.3f}')print(f'  최소: {np.min(train_intensities + val_intensities_full):.3f}')print(f'  최대: {np.max(train_intensities + val_intensities_full):.3f}')print(f'  표준편차: {np.std(train_intensities + val_intensities_full):.3f}')

증분 학습 설정
Training 파일: 5개
Validation 파일: /content/drive/MyDrive/ethic_datasets/talksets-train-6.json
총 배치 개수: 10
현재 배치 인덱스: 0
체크포인트에서 재개: 아니오 (base model부터)

[Training 데이터 로드]
데이터 로드 중: /content/drive/MyDrive/ethic_datasets/talksets-train-1.json


대화 처리 중: 100%|██████████| 19689/19689 [00:00<00:00, 239912.30it/s]


  파일 /content/drive/MyDrive/ethic_datasets/talksets-train-1.json: 70,593개 문장 로드 완료
데이터 로드 중: /content/drive/MyDrive/ethic_datasets/talksets-train-2.json


대화 처리 중: 100%|██████████| 19678/19678 [00:00<00:00, 341038.93it/s]


  파일 /content/drive/MyDrive/ethic_datasets/talksets-train-2.json: 67,904개 문장 로드 완료
데이터 로드 중: /content/drive/MyDrive/ethic_datasets/talksets-train-3.json


...

  파일 /content/drive/MyDrive/ethic_datasets/talksets-train-4.json: 67,375개 문장 로드 완료
데이터 로드 중: /content/drive/MyDrive/ethic_datasets/talksets-train-5.json


대화 처리 중: 100%|██████████| 26070/26070 [00:00<00:00, 267964.28it/s]


  파일 /content/drive/MyDrive/ethic_datasets/talksets-train-5.json: 89,265개 문장 로드 완료

총 Training 문장 수: 363,154
  - intensity > 0: 200,206개
  - intensity = 0: 162,948개

[Training 데이터를 10개 배치로 분할]
  배치 1: 36,315개 문장
  배치 2: 36,315개 문장
  배치 3: 36,315개 문장
  배치 4: 36,315개 문장
  배치 5: 36,315개 문장
  배치 6: 36,315개 문장
  배치 7: 36,315개 문장
  배치 8: 36,315개 문장
  배치 9: 36,315개 문장
  배치 10: 36,319개 문장

[Validation 데이터 로드]
데이터 로드 중: /content/drive/MyDrive/ethic_datasets/talksets-train-6.json


대화 처리 중: 100%|██████████| 13298/13298 [00:00<00:00, 236704.46it/s]


총 Validation 문장 수: 45,215

Intensity 통계:
  평균: 0.818
  최소: 0.000
  최대: 3.000
  표준편차: 0.809


In [31]:
# 현재 배치의 Training 데이터 선택
current_batch_texts, current_batch_intensities = train_batches[CURRENT_BATCH_INDEX]

print(f'\n[현재 배치 정보]')
print(f'배치 인덱스: {CURRENT_BATCH_INDEX + 1}/{NUM_BATCHES}')
print(f'Training 데이터: {len(current_batch_texts):,}개')
print(f'Validation 데이터: {len(val_texts):,}개')
print(f'\n현재 배치 Intensity 통계:')
print(f'  평균: {np.mean(current_batch_intensities):.3f}')
print(f'  최소: {np.min(current_batch_intensities):.3f}')
print(f'  최대: {np.max(current_batch_intensities):.3f}')



[현재 배치 정보]
배치 인덱스: 1/10
Training 데이터: 36,315개
Validation 데이터: 45,215개

현재 배치 Intensity 통계:
  평균: 0.823
  최소: 0.000
  최대: 3.000


## 5. Dataset 클래스 정의

PyTorch Dataset 클래스를 정의하여 talkset 데이터를 모델 학습에 사용할 수 있는 형태로 변환합니다.


In [23]:
class ImmoralityIntensityDataset(Dataset):
    """비윤리 강도 Regression을 위한 Dataset"""

    def __init__(
        self,
        texts: List[str],
        intensities: List[float],
        tokenizer,
        max_length: int = 128
    ):
        self.texts = texts
        self.intensities = intensities
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self) -> int:
        return len(self.texts)

    def __getitem__(self, idx: int) -> Dict:
        text = str(self.texts[idx])
        intensity = float(self.intensities[idx])

        # 토크나이징
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(intensity, dtype=torch.float32)
        }

print('Dataset 클래스 정의 완료')


Dataset 클래스 정의 완료


## 6. KoBERT 모델 및 토크나이저 로드

KoBERT 모델을 로드하고 Regression 태스크에 맞게 설정합니다.

**KoBERT Regression 모델 구조:**
1. KoBERT Encoder: 입력 텍스트를 벡터로 변환
2. Regression Head: 출력 레이어에서 `num_labels=1`로 설정하여 하나의 실수 값(intensity) 예측
3. Loss Function: MSE (Mean Squared Error) 사용


In [24]:
# KoBERT 모델명
MODEL_NAME = 'monologg/kobert'  # 또는 'skt/kobert-base-v1'

# 이전 체크포인트에서 모델 로드 또는 base model 로드
if RESUME_FROM_CHECKPOINT and Path(RESUME_FROM_CHECKPOINT).exists():
    print(f'[이전 체크포인트에서 모델 로드]')
    print(f'체크포인트 경로: {RESUME_FROM_CHECKPOINT}')

    tokenizer = AutoTokenizer.from_pretrained(RESUME_FROM_CHECKPOINT)
    model = AutoModelForSequenceClassification.from_pretrained(RESUME_FROM_CHECKPOINT)

    print('✓ 이전 모델 로드 완료 (증분 학습 계속)')
else:
    print(f'[Base 모델 로드]')
    print(f'모델명: {MODEL_NAME}')

    # 토크나이저 로드
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    print('✓ 토크나이저 로드 완료')

    # Regression 모델 로드 (num_labels=1: 하나의 실수 값 예측)
    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME,
        num_labels=1,  # Regression: 하나의 실수 값 출력
        problem_type="regression"  # Regression 태스크 명시
    )
    print('✓ Base 모델 로드 완료')

# GPU로 이동
model = model.to(device)
print(f'모델 장치: {device}')

# 모델 파라미터 수 확인
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'총 파라미터 수: {total_params:,}')
print(f'학습 가능한 파라미터 수: {trainable_params:,}')

# 현재 배치 정보 출력
print(f'\n[학습 정보]')
print(f'배치: {CURRENT_BATCH_INDEX + 1}/{NUM_BATCHES}')
if RESUME_FROM_CHECKPOINT:
    print(f'모드: 증분 학습 (이전 모델 기반)')
else:
    print(f'모드: 초기 학습 (Base 모델)')


[Base 모델 로드]
모델명: monologg/kobert


tokenizer_config.json:   0%|          | 0.00/263 [00:00<?, ?B/s]

The repository monologg/kobert contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/monologg/kobert .
 You can inspect the repository content at https://hf.co/monologg/kobert.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


tokenization_kobert.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/monologg/kobert:
- tokenization_kobert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

✓ 토크나이저 로드 완료


config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✓ Base 모델 로드 완료
모델 장치: cpu
총 파라미터 수: 92,187,649
학습 가능한 파라미터 수: 92,187,649

[학습 정보]
배치: 1/4
모드: 초기 학습 (Base 모델)


## 7. Dataset 객체 생성


In [25]:
# Dataset 객체 생성MAX_LENGTH = 128  # 최대 토큰 길이# 현재 배치 데이터로 Dataset 생성train_dataset = ImmoralityIntensityDataset(    current_batch_texts,    current_batch_intensities,    tokenizer,    max_length=MAX_LENGTH)# 학습 중 체크포인트용 Validation Dataset (샘플링된 데이터)val_dataset = ImmoralityIntensityDataset(    val_texts,    val_intensities,    tokenizer,    max_length=MAX_LENGTH)# 최종 검증용 전체 Validation Dataset (나중에 사용)val_dataset_full = ImmoralityIntensityDataset(    val_texts_full,    val_intensities_full,    tokenizer,    max_length=MAX_LENGTH)print(f'학습 Dataset 크기: {len(train_dataset):,} (배치 {CURRENT_BATCH_INDEX + 1}/{NUM_BATCHES})')print(f'체크포인트용 Validation Dataset 크기: {len(val_dataset):,}')print(f'최종 검증용 Validation Dataset 크기: {len(val_dataset_full):,}')

학습 Dataset 크기: 90,788 (배치 1/4)
검증 Dataset 크기: 45,215


## 8. 평가 지표 함수 정의

Regression 모델 평가를 위한 지표 함수를 정의합니다:
- MSE (Mean Squared Error)
- MAE (Mean Absolute Error)
- R² Score
- RMSE (Root Mean Squared Error)


In [ ]:
def compute_metrics(eval_pred):
    """평가 지표 계산"""
    predictions, labels = eval_pred

    # predictions는 (batch_size, 1) 형태의 배열
    # labels는 (batch_size,) 형태의 배열
    predictions = predictions.flatten()
    labels = labels.flatten()

    mse = mean_squared_error(labels, predictions)
    mae = mean_absolute_error(labels, predictions)
    r2 = r2_score(labels, predictions)

    # RMSE 계산
    rmse = np.sqrt(mse)

    return {
        'mse': mse,
        'rmse': rmse,
        'mae': mae,
        'r2': r2
    }

print('평가 지표 함수 정의 완료')


## 9. 학습 설정 (TrainingArguments)

**KoBERT 학습 로직 설명:**
1. **사전 학습된 가중치 사용**: `monologg/kobert`는 한국어 코퍼스로 사전 학습된 BERT 모델
2. **Fine-tuning**: Regression 태스크에 맞게 출력 레이어를 수정하고 전체 모델을 미세 조정
3. **Learning Rate**: 2e-5 (BERT 모델의 일반적인 학습률, 작게 설정하여 사전 학습된 가중치를 과도하게 변경하지 않음)
4. **Batch Size**: GPU 메모리에 맞게 조정 (16 또는 32)
5. **Loss Function**: MSE Loss (Regression 태스크의 표준 손실 함수)


In [ ]:
# 학습 설정OUTPUT_DIR = './kobert_immorality_intensity_regression'# 증분 학습을 위한 설정# 각 배치마다 학습할 에포크 수# 예: 10개 배치, 각 5 에포크 = 총 50 에포크 효과 (각 샘플당 평균 5회)EPOCHS_PER_BATCH = 5  # 각 배치마다 학습할 에포크 수 (3~5 권장)training_args = TrainingArguments(    output_dir=OUTPUT_DIR,    num_train_epochs=EPOCHS_PER_BATCH,  # 각 배치마다 학습할 에포크 수    per_device_train_batch_size=16,  # 학습 배치 크기    per_device_eval_batch_size=32,   # 평가 배치 크기    learning_rate=2e-5,  # 학습률 (BERT 모델의 일반적인 학습률)    weight_decay=0.01,   # L2 정규화    warmup_steps=100,    # Warmup 스텝 수 (배치 크기에 맞게 조정)    # 평가 및 저장 설정    evaluation_strategy='epoch',  # 각 에포크마다 평가    save_strategy='epoch',       # 각 에포크마다 저장    save_total_limit=5,          # 최대 체크포인트 수 (증분 학습을 위해 증가)    load_best_model_at_end=True, # 학습 종료 시 최적 모델 로드    metric_for_best_model='mse', # 최적 모델 선택 기준 (MSE 낮을수록 좋음)    greater_is_better=False,     # MSE는 낮을수록 좋으므로 False    # 체크포인트 저장 설정 (증분 학습 필수)    save_on_each_node=True,      # 각 노드에서 저장 (분산 학습 시)    save_steps=500,              # 500 스텝마다 중간 저장 (시간 제약 대비)    # 로깅 설정    logging_dir=f'{OUTPUT_DIR}/logs',    logging_steps=100,  # 100 스텝마다 로그 출력    # 기타 설정    seed=42,    fp16=True,  # Mixed precision training (GPU 메모리 절약 및 속도 향상)    dataloader_num_workers=2,  # 데이터 로딩 병렬 처리    report_to='none',  # TensorBoard 등 외부 로깅 비활성화 (필요시 변경)    # 증분 학습을 위한 추가 설정    resume_from_checkpoint=RESUME_FROM_CHECKPOINT,  # 체크포인트에서 재개    ignore_data_skip=True,  # 데이터 스킵 무시 (재개 시))print('='*60)print('학습 설정 완료')print('='*60)print(f'출력 디렉토리: {OUTPUT_DIR}')print(f'배치당 에포크 수: {EPOCHS_PER_BATCH}')print(f'현재 배치: {CURRENT_BATCH_INDEX + 1}/{NUM_BATCHES}')print(f'체크포인트에서 재개: {RESUME_FROM_CHECKPOINT if RESUME_FROM_CHECKPOINT else "아니오"}')print('='*60)

## 10. Trainer 객체 생성 및 학습

**학습 과정 설명:**
1. **Forward Pass**: 입력 텍스트를 KoBERT로 인코딩하여 벡터 표현 추출
2. **Regression Head**: 마지막 레이어에서 하나의 실수 값(intensity) 예측
3. **Loss Calculation**: 예측값과 실제 intensity 값의 MSE 계산
4. **Backward Pass**: 역전파를 통해 모델 파라미터 업데이트
5. **Evaluation**: 검증 데이터로 모델 성능 평가 (MSE, MAE, R²)


In [ ]:
# Trainer 객체 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]  # 조기 종료 (배치당 적은 에포크)
)

print('='*60)
print('Trainer 객체 생성 완료')
print('='*60)
print(f'현재 배치: {CURRENT_BATCH_INDEX + 1}/{NUM_BATCHES}')
print(f'학습 데이터: {len(train_dataset):,}개')
print(f'검증 데이터: {len(val_dataset):,}개')
print('학습 시작...')
print('='*60)

# 모델 학습
try:
    train_history = trainer.train(resume_from_checkpoint=RESUME_FROM_CHECKPOINT)

    print('\n' + '='*60)
    print('학습 완료!')
    print('='*60)
    print(f'학습 시간: {train_history.metrics["train_runtime"]:.2f}초 ({train_history.metrics["train_runtime"]/60:.1f}분)')
    print(f'총 스텝: {train_history.metrics["train_steps"]}')
    print(f'평균 Loss: {train_history.metrics["train_loss"]:.4f}')
    print('='*60)

    # 검증 결과
    val_results = trainer.evaluate()
    # 검증 결과
    val_results = trainer.evaluate()
    print(f'\n[검증 결과]')
    print(f'  MSE: {val_results["eval_mse"]:.4f}')
    print(f'  RMSE: {val_results["eval_rmse"]:.4f}')
    print(f'  MAE: {val_results["eval_mae"]:.4f}')
    print(f'  R² Score: {val_results["eval_r2"]:.4f}')

    # Google Drive에 체크포인트 복사 (선택사항)
    if USE_GOOGLE_DRIVE and DRIVE_CHECKPOINT_DIR:
        import shutil
        checkpoint_dir = Path(OUTPUT_DIR)
        checkpoint_dirs = sorted([d for d in checkpoint_dir.iterdir() if d.is_dir() and d.name.startswith('checkpoint-')])
        if checkpoint_dirs:
            latest_checkpoint = checkpoint_dirs[-1]
            drive_checkpoint_path = Path(DRIVE_CHECKPOINT_DIR) / latest_checkpoint.name

            print(f'\n[Google Drive에 체크포인트 저장]')
            print(f'  원본: {latest_checkpoint}')
            print(f'  대상: {drive_checkpoint_path}')

            # 이전 체크포인트 삭제 후 복사
            if drive_checkpoint_path.exists():
                shutil.rmtree(drive_checkpoint_path)
            shutil.copytree(latest_checkpoint, drive_checkpoint_path)

            print(f'✓ Drive 저장 완료')

except KeyboardInterrupt:
    print('\n⚠️ 학습이 중단되었습니다.')
    print(f'현재 배치 {CURRENT_BATCH_INDEX + 1}의 체크포인트가 저장되었습니다.')
    print('다음 세션에서 RESUME_FROM_CHECKPOINT를 설정하여 이어서 학습하세요.')
except Exception as e:
    print(f'\n❌ 학습 중 오류 발생: {e}')
    print(f'현재 배치 {CURRENT_BATCH_INDEX + 1}의 체크포인트가 저장되었을 수 있습니다.')
    print('체크포인트를 확인하고 RESUME_FROM_CHECKPOINT를 설정하여 재시도하세요.')


## 11. 현재 배치 학습 완료 및 다음 단계 안내

현재 배치 학습이 완료되었습니다. 다음 단계를 진행하세요.


In [ ]:
# 현재 배치 학습 완료 후 모델 저장
print('='*60)
print('현재 배치 학습 완료')
print('='*60)

# 최적 모델의 체크포인트 경로 찾기
checkpoint_dir = Path(OUTPUT_DIR)
checkpoint_dirs = sorted([d for d in checkpoint_dir.iterdir() if d.is_dir() and d.name.startswith('checkpoint-')])
if checkpoint_dirs:
    latest_checkpoint = checkpoint_dirs[-1]
    print(f'✓ 최신 체크포인트: {latest_checkpoint}')

    # 다음 배치를 위한 정보 저장
    next_batch_info = {
        'current_batch_index': CURRENT_BATCH_INDEX,
        'total_batches': NUM_BATCHES,
        'latest_checkpoint': str(latest_checkpoint),
        'validation_results': {
            'mse': float(val_results['eval_mse']),
            'rmse': float(val_results['eval_rmse']),
            'mae': float(val_results['eval_mae']),
            'r2': float(val_results['eval_r2'])
        }
    }

    info_file = checkpoint_dir / f'batch_{CURRENT_BATCH_INDEX}_info.json'
    with open(info_file, 'w', encoding='utf-8') as f:
        json.dump(next_batch_info, f, indent=2, ensure_ascii=False)

    print(f'✓ 배치 정보 저장: {info_file}')

    # Google Drive에도 배치 정보 저장 (선택사항)
    if USE_GOOGLE_DRIVE and DRIVE_CHECKPOINT_DIR:
        import shutil
        drive_info_file = Path(DRIVE_CHECKPOINT_DIR) / f'batch_{CURRENT_BATCH_INDEX}_info.json'
        shutil.copy(info_file, drive_info_file)
        print(f'✓ Drive에도 배치 정보 저장: {drive_info_file}')

# 다음 배치 정보 출력
if CURRENT_BATCH_INDEX < NUM_BATCHES - 1:
    print(f'\n[다음 단계]')
    print(f'배치 {CURRENT_BATCH_INDEX + 1}/{NUM_BATCHES} 완료')
    print(f'다음 배치 ({CURRENT_BATCH_INDEX + 2}/{NUM_BATCHES})를 학습하려면:')
    print(f'  1. CURRENT_BATCH_INDEX = {CURRENT_BATCH_INDEX + 1}로 설정')
    print(f'  2. RESUME_FROM_CHECKPOINT = "{latest_checkpoint}"로 설정')
    print(f'  3. 노트북을 다시 실행하세요')
else:
    print(f'\n[모든 배치 학습 완료!]')
    print(f'총 {NUM_BATCHES}개 배치 학습 완료')
    print(f'최종 모델: {latest_checkpoint}')

print('='*60)


## 11. 테스트 데이터 평가


In [ ]:
# 테스트 데이터로 최종 평가
print('테스트 데이터 평가 중...')
test_results = trainer.evaluate(test_dataset)

print('\n=== 테스트 결과 ===')
print(f'MSE: {test_results["eval_mse"]:.4f}')
print(f'RMSE: {test_results["eval_rmse"]:.4f}')
print(f'MAE: {test_results["eval_mae"]:.4f}')
print(f'R² Score: {test_results["eval_r2"]:.4f}')


## 12. 예측 예시

학습된 모델로 새로운 텍스트의 비윤리 강도를 예측합니다.


In [ ]:
# 학습된 모델로 예측
def predict_intensity(model, tokenizer, text: str, device) -> float:
    """단일 텍스트의 비윤리 강도 예측"""
    model.eval()

    # 토크나이징
    encoding = tokenizer(
        text,
        truncation=True,
        padding='max_length',
        max_length=128,
        return_tensors='pt'
    )

    # GPU로 이동
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    # 예측
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predicted_intensity = outputs.logits.item()

    return max(0.0, predicted_intensity)  # 음수 방지

# 예측 예시
sample_texts = [
    "안녕하세요, 문의드리고 싶은 것이 있습니다.",
    "지금 당장 해결해달라니깐!",
    "원래 틀딱들은 눈치가 없어서 ㅋㅋㅋ",
    "해결 못하면 고소하겠습니다."
]

print('=== 예측 예시 ===')
for text in sample_texts:
    predicted = predict_intensity(model, tokenizer, text, device)
    print(f'\n발화: {text}')
    print(f'예측된 비윤리 강도: {predicted:.3f}')


## 13. 모델 저장

학습된 모델과 토크나이저를 저장합니다.


In [ ]:
# 최종 모델 저장 (모든 배치 학습 완료 후)if CURRENT_BATCH_INDEX == NUM_BATCHES - 1:    print('='*60)    print('최종 모델 저장')    print('='*60)    # 최신 체크포인트에서 최종 모델 로드    checkpoint_dir = Path(OUTPUT_DIR)    checkpoint_dirs = sorted([d for d in checkpoint_dir.iterdir() if d.is_dir() and d.name.startswith('checkpoint-')])    if checkpoint_dirs:        latest_checkpoint = checkpoint_dirs[-1]        print(f'최신 체크포인트에서 로드: {latest_checkpoint}')        final_model = AutoModelForSequenceClassification.from_pretrained(str(latest_checkpoint))        final_tokenizer = AutoTokenizer.from_pretrained(str(latest_checkpoint))                # 최종 Trainer 생성 (전체 validation 데이터로 평가)        final_trainer = Trainer(            model=final_model,            args=training_args,            compute_metrics=compute_metrics        )    else:        print('체크포인트를 찾을 수 없습니다. 현재 모델을 사용합니다.')        final_model = model        final_tokenizer = tokenizer        final_trainer = trainer        # ===== 최종 검증 (전체 Validation 데이터 사용) =====    print('\n' + '='*60)    print('최종 검증 (전체 Validation 데이터)')    print('='*60)    print(f'최종 검증 데이터 크기: {len(val_dataset_full):,}개')    print('평가 중...')        final_val_results = final_trainer.evaluate(val_dataset_full)        print(f'\n[최종 검증 결과]')    print(f'  MSE: {final_val_results["eval_mse"]:.4f}')    print(f'  RMSE: {final_val_results["eval_rmse"]:.4f}')    print(f'  MAE: {final_val_results["eval_mae"]:.4f}')    print(f'  R² Score: {final_val_results["eval_r2"]:.4f}')    print('='*60)    # 최종 모델 저장    final_model_path = f'{OUTPUT_DIR}/final_model'    final_model.save_pretrained(final_model_path)    final_tokenizer.save_pretrained(final_model_path)    print(f'✓ 최종 모델이 저장되었습니다: {final_model_path}')    # 평가 결과 저장    results_summary = {        'incremental_training': {            'total_batches': NUM_BATCHES,            'epochs_per_batch': EPOCHS_PER_BATCH,            'train_files': TRAIN_DATA_FILES,            'val_file': VAL_DATA_FILE        },        'checkpoint_validation': {            'mse': float(val_results['eval_mse']),            'rmse': float(final_val_results['eval_rmse']),            'mae': float(final_val_results['eval_mae']),            'r2': float(final_val_results['eval_r2']),            'sample_size': len(val_texts)        },        'final_validation': {            'mse': float(final_val_results['eval_mse']),            'rmse': float(final_val_results['eval_rmse']),            'mae': float(final_val_results['eval_mae']),            'r2': float(final_val_results['eval_r2']),            'sample_size': len(val_texts_full)        },        'training_info': {            'total_train_samples': len(train_texts),            'checkpoint_val_samples': len(val_texts),            'final_val_samples': len(val_texts_full),            'model_name': MODEL_NAME,            'max_length': MAX_LENGTH,            'final_checkpoint': str(latest_checkpoint) if checkpoint_dirs else None        }    }    # 결과 요약 저장    results_file = f'{OUTPUT_DIR}/evaluation_results.json'    with open(results_file, 'w', encoding='utf-8') as f:        json.dump(results_summary, f, indent=2, ensure_ascii=False)    print(f'✓ 평가 결과가 저장되었습니다: {results_file}')    print('\n' + '='*60)    print('📦 저장된 파일 목록:')    print(f'  1. 최종 모델: {final_model_path}/')    print(f'     - pytorch_model.bin')    print(f'     - config.json')    print(f'     - tokenizer 관련 파일들')    print(f'  2. 평가 결과: {results_file}')    print(f'  3. 체크포인트들: {OUTPUT_DIR}/checkpoint-*/')    print('='*60)else:    print('\n[주의]')    print(f'현재 배치 {CURRENT_BATCH_INDEX + 1}/{NUM_BATCHES}만 완료되었습니다.')    print('모든 배치 학습 완료 후 최종 모델이 자동으로 저장됩니다.')    print('다음 배치를 학습하려면 위의 안내를 따르세요.')print('\n📖 모델 적용 방법:')print('   프로젝트의 MODEL_APPLICATION_GUIDE.md 파일을 참조하세요.')

## 15. 결과 파일 다운로드 (Colab)

Colab에서 학습한 모델을 로컬로 다운로드하는 방법입니다.

**주의**: Colab 세션이 종료되면 파일이 삭제되므로, 반드시 다운로드하거나 Google Drive에 저장하세요.


## Google Drive 연동 (선택사항, 권장)

**장점:**
- 체크포인트가 자동으로 Drive에 저장되어 안전
- 다음 세션에서 자동으로 로드 가능
- Colab 세션이 종료되어도 데이터 유지

**설정 방법:**
1. 아래 셀 실행하여 Drive 마운트
2. `USE_GOOGLE_DRIVE = True` 설정
3. 체크포인트가 자동으로 Drive에 저장됨


In [ ]:
# Google Drive 연동 설정 (선택사항)
USE_GOOGLE_DRIVE = False  # True로 설정하면 Drive에 저장

if USE_GOOGLE_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive')

    # Drive에 체크포인트 저장 경로 설정
    DRIVE_CHECKPOINT_DIR = '/content/drive/MyDrive/kobert_immorality_checkpoints'
    Path(DRIVE_CHECKPOINT_DIR).mkdir(parents=True, exist_ok=True)

    # 기존 체크포인트 자동 로드 (가장 최신 것)
    import glob
    existing_checkpoints = sorted(glob.glob(f'{DRIVE_CHECKPOINT_DIR}/checkpoint-*'))
    if existing_checkpoints and RESUME_FROM_CHECKPOINT is None:
        latest_checkpoint = existing_checkpoints[-1]
        print(f'[Drive에서 최신 체크포인트 발견]')
        print(f'  경로: {latest_checkpoint}')

        # 배치 정보 확인
        batch_info_file = Path(DRIVE_CHECKPOINT_DIR) / f'batch_{CURRENT_BATCH_INDEX - 1}_info.json'
        if batch_info_file.exists():
            with open(batch_info_file, 'r', encoding='utf-8') as f:
                batch_info = json.load(f)
            print(f'  마지막 배치: {batch_info["current_batch_index"] + 1}/{batch_info["total_batches"]}')

            # 자동으로 다음 배치 설정 제안
            if batch_info['current_batch_index'] < batch_info['total_batches'] - 1:
                print(f'  [제안] 다음 배치로 진행하려면:')
                print(f'    CURRENT_BATCH_INDEX = {batch_info["current_batch_index"] + 1}')
                print(f'    RESUME_FROM_CHECKPOINT = "{latest_checkpoint}"')

        RESUME_FROM_CHECKPOINT = latest_checkpoint
        print()
else:
    DRIVE_CHECKPOINT_DIR = None
    print('[Google Drive 사용 안 함]')
    print('체크포인트는 로컬에만 저장됩니다.')
    print('각 세션 후 수동으로 다운로드하거나 Drive에 저장하세요.')


In [ ]:
# 방법 1: 압축 파일로 다운로드
from google.colab import files
import shutil

# 모델 폴더 압축
zip_path = 'kobert_immorality_model.zip'
shutil.make_archive('kobert_immorality_model', 'zip', final_model_path)

print(f'✓ 압축 파일 생성: {zip_path}')
print('다운로드 중...')
files.download(zip_path)

# 평가 결과도 다운로드
files.download(results_file)

print('\n✓ 모든 파일 다운로드 완료!')


### Google Drive에 저장하는 방법 (선택사항)

Google Drive에 저장하면 나중에 다시 사용할 수 있습니다.


In [ ]:
# 방법 2: Google Drive에 저장 (선택사항)
# 주석을 해제하여 사용하세요

# from google.colab import drive
# drive.mount('/content/drive')

# # Drive에 복사
# import shutil
# drive_model_path = '/content/drive/MyDrive/kobert_immorality_intensity_model'
# shutil.copytree(final_model_path, drive_model_path, dirs_exist_ok=True)
# shutil.copy(results_file, '/content/drive/MyDrive/evaluation_results.json')

# print(f'✓ 모델이 Google Drive에 저장되었습니다: {drive_model_path}')


## 증분 학습 가이드

### 증분 학습 전략

**문제점:**
- Colab 무료 버전은 최대 12시간 런타임 제한
- 전체 데이터를 한 번에 학습하면 시간 부족

**해결책:**
1. 전체 데이터를 여러 배치(예: 4개)로 나누기
2. 각 배치마다 이전 모델을 로드하여 계속 fine-tuning
3. 각 배치 학습 후 체크포인트 저장
4. 다음 세션에서 마지막 체크포인트 로드하여 이어서 학습

### 사용 방법

**1단계: 첫 번째 배치 학습**
```python
CURRENT_BATCH_INDEX = 0
RESUME_FROM_CHECKPOINT = None
```
- 노트북 실행
- 배치 0 학습 완료 후 체크포인트 경로 확인

**2단계: 두 번째 배치 학습 (새로운 Colab 세션)**
```python
CURRENT_BATCH_INDEX = 1
RESUME_FROM_CHECKPOINT = './kobert_immorality_intensity_regression/checkpoint-XXXX'
```
- 노트북 다시 실행
- 이전 모델을 로드하여 배치 1 학습

**3단계: 나머지 배치 반복**
- CURRENT_BATCH_INDEX를 2, 3, ... 으로 증가
- RESUME_FROM_CHECKPOINT를 최신 체크포인트로 업데이트
- 각 세션마다 반복

### 체크포인트 저장 위치

- Colab 로컬: `./kobert_immorality_intensity_regression/checkpoint-*/`
- Google Drive 권장: 각 세션 후 Drive에 복사

### 주의사항

1. **각 세션 후 체크포인트 다운로드 필수** (또는 Drive에 저장)
2. **배치 순서 유지**: 반드시 순차적으로 학습 (0 → 1 → 2 → 3)
3. **배치 개수 조정**: 시간에 따라 NUM_BATCHES 변경 가능 (3~5 권장)
4. **에포크 수 조정**: EPOCHS_PER_BATCH를 변경하여 전체 학습량 조절


## 학습 요약

### KoBERT Regression 모델 학습 로직:

1. **모델 구조**:
   - KoBERT Encoder (12 layers, 768 hidden size)
   - Regression Head (Linear layer: 768 → 1)

2. **학습 방법**:
   - **사전 학습된 가중치 활용**: `monologg/kobert`는 한국어로 사전 학습된 BERT 모델
   - **Fine-tuning**: Regression 태스크에 맞게 출력 레이어만 추가하고 전체 모델 미세 조정
   - **학습률**: 2e-5 (작은 값으로 사전 학습된 가중치를 서서히 조정)
   - **Loss**: MSE (Mean Squared Error)

3. **데이터 흐름**:
   - 입력: 발화 텍스트 → KoBERT 토크나이저 → 토큰 IDs
   - KoBERT 인코딩: 토큰 IDs → 벡터 표현 (768차원)
   - Regression: 벡터 표현 → Linear Layer → intensity 값 (실수)
   - Loss: 예측 intensity vs 실제 intensity의 MSE

4. **최적화**:
   - AdamW Optimizer (BERT 모델의 표준 옵티마이저)
   - Learning Rate Warmup (초기 학습률을 점진적으로 증가)
   - Early Stopping (검증 성능이 개선되지 않으면 조기 종료)

5. **평가 지표**:
   - MSE: 평균 제곱 오차 (낮을수록 좋음)
   - RMSE: 평균 제곱근 오차 (실제 intensity 단위)
   - MAE: 평균 절대 오차 (실제 intensity 단위)
   - R²: 결정계수 (1에 가까울수록 좋음)
